<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.0
    Uninstalling protobuf-6.32.0:
      Successfully uninstalled protobuf-6.32.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-08-20 11:02:35
-------------------
qualified stocks: 93
with latest results: 27
still star stocks: 17
-------------------
Initial Investment:  1.20 C
CY Investment:  1.47 C
Reserve:  3.39 L
Current:  1.38 C
-------------------
Today PnL: 71.31 K (0.52%)
Current PnL: -17.01 L (-11.57%)
CY Booked + Current PnL: -8.28 L (-5.63%)
-------------------
Total profit:  3.77 L
Total loss:  -20.78 L
-------------------
Total Booked + Current PnL: 18.70 L (15.58%)
Total Booked PnL: 35.71 L (29.75%)
Curr Year Booked PnL: 8.73 L (6.31%)
Prev Year Booked PnL: 26.98 L (22.48%)
Est FTT:  2.14 C
Est FTT PnL: 76.15 L (55.05%)
Deployed:  1.20 C
Current:  1.38 C
CAGR/XIRR %: 9.07%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
37,HINDUNILVR,2413.81,2723.00,-5.60,H-LC,6.55,296800.0,26453.0,8162.0,1.74,9.78,2.75,12.81,24.0,3.24,2.20,24.13,XY25,NTT,FMCG
4,APOLLOHOSP,7098.95,8285.00,-12.98,H-LC,6.99,173800.0,17623.0,8464.0,0.98,11.28,4.87,16.71,22.0,2.08,1.29,30.69,X40N,BTT,HEALTHCARE
68,SBIN,760.30,863.00,-11.34,M-LC,3.76,220056.0,18576.0,8648.0,0.33,9.22,3.93,13.51,60.0,2.15,1.63,22.98,XY25,NTT,BANKS
36,HINDALCO,651.95,756.01,-18.25,H-LC,6.92,112296.0,7984.0,8669.0,-0.69,7.65,7.72,15.96,9.0,0.92,0.83,25.84,X5K,ATH,METALS
55,MEDANTA,1087.93,1486.00,13.12,H-MC,14.29,161507.0,35307.0,10869.0,1.38,27.98,6.73,36.59,108.0,3.25,1.20,44.00,XY24,NTT,HEALTHCARE


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BANKINDIA,116.91,190.00,-30.43,H-MC,7.44,179592.0,-216.0,112622.0,0.24,-0.12,62.71,62.52,88.0,-0.00,1.33,33.93,XR,NTT,BANKS
58,PGHH,13388.00,18062.58,-31.08,H-MC,5.97,199515.0,-1305.0,71426.0,0.05,-0.65,35.80,34.92,80.0,-0.02,1.48,4.00,X40,ATH,FMCG


In [11]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,ITC,411.90,452.0,-37.92,H-LC,3.02,148876.0,-1056.0,15647.0,-0.02,-0.70,10.51,9.74,4.0,-0.07,1.10,5.49,X40,NTT,FMCG
21,CIPLA,1492.70,1795.0,-21.15,H-LC,3.24,210802.0,6302.0,35120.0,-0.66,3.08,16.66,20.25,10.0,0.18,1.56,12.08,X40N,BTT,PHARMA
53,LTIM,5564.16,7230.2,-5.78,H-LC,3.65,186840.0,-13470.0,73447.0,1.55,-6.72,39.31,29.94,16.0,-0.18,1.38,29.08,X200,ATH,IT
57,NESTLEIND,1134.30,1377.0,-10.77,H-LC,3.73,274739.0,9313.0,47475.0,1.09,3.51,17.28,21.40,11.0,0.20,2.04,10.39,XY25,NTT,FMCG
5,ASIANPAINT,2961.56,4250.0,-10.32,H-LC,4.74,174896.0,-26490.0,114102.0,-0.29,-13.15,65.24,43.51,27.0,-0.23,1.30,21.75,X40,BTT,PAINTS


In [12]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
29,EASEMYTRIP,18.12,26.40,-3.48,M-SC,26.26,92994.0,-83821.0,164618.0,5.19,-47.41,177.02,45.70,197.0,-0.51,0.69,8.17,XY24,NTT,TRAVEL
6,ASIANTILES,75.41,137.00,7322.22,L-SC,7.08,81457.0,-12353.0,88967.0,4.37,-13.17,109.22,81.67,269.0,-0.14,0.60,57.75,XR,NTT,CERAMICS
61,RAJOOENG,102.50,143.33,-29.34,H-SC,7.92,107150.0,4650.0,36185.0,3.87,4.54,33.77,39.83,114.0,0.13,0.79,6.09,AR,ATH,MISC
23,COFFEEDAY,59.14,80.00,-47.69,L-SC,22.03,78470.0,-35079.0,75127.0,3.68,-30.89,95.74,35.27,268.0,-0.47,0.58,92.06,XR,NTT,HOTELS
45,INFY,1461.46,2275.00,-18.88,H-LC,11.81,265010.0,3409.0,142204.0,2.81,1.30,53.66,55.67,3.0,0.02,1.96,7.49,X40,BTT,IT


In [13]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,KPIGREEN,497.21,731.35,10.94,H-SC,11.28,132023.0,6726.0,52281.0,-1.81,5.37,39.60,47.09,141.0,0.13,0.98,65.24,MH,ATH,POWER
79,TATAMOTORS,761.77,1065.00,-20.84,M-LC,4.97,250270.0,-26253.0,136322.0,-1.54,-9.49,54.47,39.81,54.0,-0.19,1.85,19.94,XY24,NTT,AUTO
88,VALIANTORG,512.64,838.00,-320.21,H-SC,15.42,93509.0,-39777.0,124367.0,-1.53,-29.84,133.00,63.47,139.0,-0.32,0.69,56.74,XR,NTT,CHEMICALS
52,LAOPALA,369.40,464.00,119.62,H-SC,2.12,69752.0,-31094.0,56918.0,-1.48,-30.83,81.60,25.61,142.0,-0.55,0.52,29.32,AR,NTT,CERAMICS
9,BAJAJFINSV,1994.80,2758.00,-10.95,H-LC,7.00,95599.0,-2146.0,39540.0,-1.07,-2.20,41.36,38.26,35.0,-0.05,0.71,25.18,X40,BTT,FINANCE


In [14]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,TTKPRESTIG,769.29,770.0,100.00,M-SC,9.60,86211.0,-14566.0,14656.0,-0.84,-14.45,17.00,0.09,245.0,-0.99,0.64,13.26,OX40N,NTT,DURABLES
43,INDIGOPNTS,1407.73,1408.0,122.66,M-SC,1.90,146060.0,-28499.0,28526.0,0.56,-16.33,19.53,0.02,221.0,-1.00,1.08,26.66,OX40N,NTT,PAINTS
84,UJJIVANSFB,52.77,60.0,86.70,H-SC,16.49,117828.0,-24651.0,44174.0,1.58,-17.30,37.49,13.70,163.0,-0.56,0.87,39.07,OX40N,NTT,BANKS
20,CERA,7989.07,9475.0,-17.90,H-SC,2.99,117270.0,-26533.0,53276.0,0.25,-18.45,45.43,18.60,149.0,-0.50,0.87,29.41,OX40N,NTT,CERAMICS
50,KANSAINER,299.63,340.0,-68.52,H-SC,3.25,216972.0,-52695.0,89024.0,-0.74,-19.54,41.03,13.47,138.0,-0.59,1.61,9.74,XY24,NTT,PAINTS


In [15]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
80,TCS,3794.03,4998.00,-28.04,H-LC,16.22,238033.0,-57901.0,151817.0,1.18,-19.57,63.78,31.73,1.0,-0.38,1.76,1.62,X40,BTT,IT
45,INFY,1461.46,2275.00,-18.88,H-LC,11.81,265010.0,3409.0,142204.0,2.81,1.30,53.66,55.67,3.0,0.02,1.96,7.49,X40,BTT,IT
47,ITC,411.90,452.00,-37.92,H-LC,3.02,148876.0,-1056.0,15647.0,-0.02,-0.70,10.51,9.74,4.0,-0.07,1.10,5.49,X40,NTT,FMCG
89,VBL,492.64,671.64,-8.45,H-LC,9.62,278146.0,13106.0,83193.0,2.30,4.94,29.91,36.33,5.0,0.16,2.06,18.90,X40N,ATH,FMCG
1,ABB,5551.76,7934.00,-41.13,H-LC,11.62,199875.0,-16644.0,109551.0,1.12,-7.69,54.81,42.91,7.0,-0.15,1.48,4.68,AR,NTT,ELECTRICAL


In [16]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.0,13.12,H-MC,14.29,161507.0,35307.0,10869.0,1.38,27.98,6.73,36.59,108.0,3.25,1.20,44.00,XY24,NTT,HEALTHCARE
28,DMART,3764.92,5201.0,-7.44,H-LC,7.96,231158.0,46677.0,23694.0,1.38,25.30,10.25,38.14,38.0,1.97,1.71,38.59,X40N,NTT,FMCG


In [17]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.00,13.12,H-MC,14.29,161507.0,35307.0,10869.0,1.38,27.98,6.73,36.59,108.0,3.25,1.20,44.00,XY24,NTT,HEALTHCARE
15,BLUESTARCO,1646.70,2326.38,12.85,H-MC,5.92,194800.0,30130.0,37830.0,1.00,18.30,19.42,41.28,89.0,0.80,1.44,27.74,X40N,ATH,AC
51,KPIGREEN,497.21,731.35,10.94,H-SC,11.28,132023.0,6726.0,52281.0,-1.81,5.37,39.60,47.09,141.0,0.13,0.98,65.24,MH,ATH,POWER
90,VOLTAS,1278.28,1918.49,-1.78,H-MC,7.39,206055.0,14313.0,81721.0,-0.38,7.46,39.66,50.08,99.0,0.18,1.53,14.77,XY25,ATH,AC
35,HEROMOTOCO,4311.81,5949.07,-3.00,H-MC,10.46,180142.0,29229.0,28084.0,0.56,19.37,15.59,37.97,93.0,1.04,1.34,49.57,AR,ATH,AUTO


In [18]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,HEROMOTOCO,4311.81,5949.07,-3.00,H-MC,10.46,180142.0,29229.0,28084.0,0.56,19.37,15.59,37.97,93.0,1.04,1.34,49.57,AR,ATH,AUTO
61,RAJOOENG,102.50,143.33,-29.34,H-SC,7.92,107150.0,4650.0,36185.0,3.87,4.54,33.77,39.83,114.0,0.13,0.79,6.09,AR,ATH,MISC
42,INDIAMART,2327.09,4850.92,-50.16,H-SC,11.95,139125.0,15789.0,117978.0,1.37,12.80,84.80,108.45,119.0,0.13,1.03,38.02,AR,ATH,MISC
51,KPIGREEN,497.21,731.35,10.94,H-SC,11.28,132023.0,6726.0,52281.0,-1.81,5.37,39.60,47.09,141.0,0.13,0.98,65.24,MH,ATH,POWER
91,WHIRLPOOL,1219.98,2270.00,-42.43,M-SC,4.99,97402.0,5904.0,72847.0,1.05,6.45,74.79,86.07,223.0,0.08,0.72,40.90,XR,NTT,DURABLES


In [19]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
23,COFFEEDAY,59.14,80.00,-47.69,L-SC,22.03,78470.0,-35079.0,75127.0,3.68,-30.89,95.74,35.27,268.0,-0.47,0.58,92.06,XR,NTT,HOTELS
6,ASIANTILES,75.41,137.00,7322.22,L-SC,7.08,81457.0,-12353.0,88967.0,4.37,-13.17,109.22,81.67,269.0,-0.14,0.60,57.75,XR,NTT,CERAMICS
3,ALKYLAMINE,2347.98,4546.37,-2.49,H-SC,15.60,90704.0,-10259.0,104790.0,0.25,-10.16,115.53,93.63,148.0,-0.10,0.67,35.10,SR,ATH,CHEMICALS
54,MASFIN,326.60,399.50,-16.44,H-SC,12.42,94800.0,-3180.0,25046.0,0.85,-3.25,26.42,22.32,152.0,-0.13,0.70,37.49,XR,ATH,FINANCE
36,HINDALCO,651.95,756.01,-18.25,H-LC,6.92,112296.0,7984.0,8669.0,-0.69,7.65,7.72,15.96,9.0,0.92,0.83,25.84,X5K,ATH,METALS


In [20]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,HEROMOTOCO,4311.81,5949.07,-3.00,H-MC,10.46,180142.0,29229.0,28084.0,0.56,19.37,15.59,37.97,93.0,1.04,1.34,49.57,AR,ATH,AUTO
28,DMART,3764.92,5201.00,-7.44,H-LC,7.96,231158.0,46677.0,23694.0,1.38,25.30,10.25,38.14,38.0,1.97,1.71,38.59,X40N,NTT,FMCG
4,APOLLOHOSP,7098.95,8285.00,-12.98,H-LC,6.99,173800.0,17623.0,8464.0,0.98,11.28,4.87,16.71,22.0,2.08,1.29,30.69,X40N,BTT,HEALTHCARE
5,ASIANPAINT,2961.56,4250.00,-10.32,H-LC,4.74,174896.0,-26490.0,114102.0,-0.29,-13.15,65.24,43.51,27.0,-0.23,1.30,21.75,X40,BTT,PAINTS
37,HINDUNILVR,2413.81,2723.00,-5.60,H-LC,6.55,296800.0,26453.0,8162.0,1.74,9.78,2.75,12.81,24.0,3.24,2.20,24.13,XY25,NTT,FMCG


In [21]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.75
1,25,41.35
2,50,71.01


In [22]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    37.46
LC    36.04
MC    26.45
Name: CurrAlloc%, dtype: float64

In [23]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.60
X40      14.29
X40N     13.87
XY25     11.85
XR        9.01
AR        8.44
OX40N     7.44
X5K       2.22
MH        1.73
X200      1.38
SR        1.12
Name: CurrAlloc%, dtype: float64

In [24]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    27.81
H-MC    23.52
H-SC    22.62
M-SC    13.30
M-LC     7.17
M-MC     2.60
L-SC     1.54
L-LC     1.06
L-MC     0.33
Name: CurrAlloc%, dtype: float64

In [25]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.54,0.97,29.35
IT,11.81,-17.07,75.53
FINANCE,8.12,-19.02,63.05
BANKS,7.57,-11.62,59.01
MISC,5.84,-16.59,83.42
PAINTS,5.64,-14.74,37.35
ELECTRICAL,4.72,-10.23,57.97
AUTO,4.57,-9.44,57.05
AC,3.70,2.99,29.86


In [26]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2776654.0,23
XR,1074764.0,13
AR,1069170.0,10
X40,807568.0,11
X40N,495063.0,11
OX40N,453521.0,10
XY25,450829.0,8
SR,192698.0,2
X5K,139267.0,2


In [27]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2721155.0,27
M-SC,1451113.0,17
H-MC,1203265.0,18
H-LC,1147834.0,19
M-LC,410435.0,5
M-MC,326232.0,2
L-SC,252339.0,3
L-MC,58033.0,1
L-LC,44180.0,1


In [28]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,922967.0,6
M-SC,XY24,852280.0,7
H-SC,AR,652577.0,5
H-LC,X40,520269.0,6
H-SC,XR,513986.0,6
H-MC,XY24,469343.0,5
M-MC,XY24,326232.0,2
H-LC,AR,230560.0,2
H-MC,X40,207166.0,4


In [29]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 30.0 seconds
